In [4]:
import numpy as np
import pandas as pd

data = pd.read_csv("spetsialistidVsTurvaintsidendid2.csv")
data2 = data.copy(deep=True)
data2 = data.drop(['Tööga hõivatud isikute arv', 'Tegevusala', 'Näitaja', 'Tegevusala (EMTAK 2008)', 'Turvalisus'], axis=1)
data2

testDistribution = []
for i in range(10000):
    for column in data2.columns:
        np.random.shuffle(data2[column])
    pairwise_correlations = data2.corr()
    pairwise_correlations.values[np.triu_indices_from(pairwise_correlations, 0)] = np.nan
    maxVal = np.max(np.abs(pairwise_correlations.stack()))
    testDistribution.append(maxVal)
    
sigThreshold = np.max(testDistribution)
print(f"threshold: {sigThreshold}")

data3 = data.copy(deep=True)
pairwise_correlations = data3.corr()
pairwise_correlations


threshold: 0.7130428369735771


,Value,Value.1
Value,1.000000,-0.694316
Value.1,-0.694316,1.000000


In [ ]:
%%time
import numpy as np
import pandas as pd

data = pd.read_csv("kokku.csv")
#data = data.loc[data['Value.1'] == 30.2]
cs = ["Tööga hõivatud isikute arv", "Näitaja", "Turvalisus"] # columns
uvs = [list(pd.unique(data[c])) for c in cs] # unique values
#uvs[1].remove("Kokku – kõik tegevusalad")
corrs = []

def jobs():
    for v0 in uvs[0]:
        for v1 in uvs[1]:
            for v2 in uvs[2]:
                yield v0, v1, v2

for job in jobs(): #list(jobs())[:10]:
    tmp = data.loc[(data[cs[0]] == job[0]) \
                   & (data[cs[1]] == job[1]) \
                   & (data[cs[2]] == job[2])]
    tmp = tmp.loc[tmp["Tegevusala"] != "Kokku – kõik tegevusalad"]
    tmp = tmp.filter(items=["Value", "Value.1"])
    corrs.append((job, tmp.corr().iloc[0, 1]))
corrs

In [ ]:
[[*e[0], e[1]] for e in corrs if e[1] > 0.90]

**Getting the numbers of different companies from the Estonian Tax and Customs Board
(Eesti Maksu- ja Tolliamet) dataset and grouping them by size and by fields of activity
using Jupyter Notebook: 5h (Tõnis Rahe).**

In [44]:
import numpy as np
import pandas as pd
taxes_data = pd.read_csv("tasutud_maksud_2019_iv_kvartal.csv", sep=';')
#uniquev = taxes_data['tegevusvaldkond'].unique()
#print(uniquev)
taxes_data['Tootajaid'] = pd.to_numeric(taxes_data['Tootajaid'],errors='coerce')
taxes_data = taxes_data.replace(np.nan, 0, regex=True)
taxes_data['Tootajaid'] = taxes_data['Tootajaid'].astype(int)
taxes_data.groupby(['tegevusvaldkond']).agg({'Registrikood': 'count', 'Tootajaid': 'sum'})
#sum_df= taxes_data.groupby(['tegevusvaldkond']).agg({'Nimi'})

,Registrikood,Tootajaid
tegevusvaldkond,,
0,3580,2977
AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS,257,41632
EHITUS,14080,51667
EKSTERRITORIAALSETE ORGANISATSIOONIDE JA ÜKSUSTE TEGEVUS,2,11
"ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE",392,5150
FINANTS- JA KINDLUSTUSTEGEVUS,2104,13962
HALDUS- JA ABITEGEVUSED,7428,43964
HARIDUS,3513,67837
HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT,21127,96727


In [43]:
#Number of employees in the enterprises: 10-19
filtered1 = taxes_data[(taxes_data['Tootajaid'] > 9) & (taxes_data['Tootajaid'] < 20) ][['tegevusvaldkond','Nimi','Tootajaid']]
filtered1.groupby(['tegevusvaldkond']).agg({'Nimi':'count','Tootajaid':'sum'})


,Nimi,Tootajaid
tegevusvaldkond,,
0,14,191
AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS,18,256
EHITUS,781,10329
EKSTERRITORIAALSETE ORGANISATSIOONIDE JA ÜKSUSTE TEGEVUS,1,10
"ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE",17,227
FINANTS- JA KINDLUSTUSTEGEVUS,59,775
HALDUS- JA ABITEGEVUSED,243,3307
HARIDUS,316,4522
HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT,769,10175


In [38]:
#Number of employees in the enterprises: 20-49
filtered2 = taxes_data[(taxes_data['Tootajaid'] > 19) & (taxes_data['Tootajaid'] < 50) ][['tegevusvaldkond','Nimi','Tootajaid']]
filtered2.groupby(['tegevusvaldkond']).agg({'Nimi':'count','Tootajaid':'sum'})


,Nimi,Tootajaid
tegevusvaldkond,,
0,10,286
AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS,39,1314
EHITUS,287,8118
"ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE",20,602
FINANTS- JA KINDLUSTUSTEGEVUS,40,1164
HALDUS- JA ABITEGEVUSED,176,5282
HARIDUS,604,19727
HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT,365,11140
INFO JA SIDE,102,3072


In [39]:
#Number of employees in the enterprises: 50-99
filtered3 = taxes_data[(taxes_data['Tootajaid'] > 49) & (taxes_data['Tootajaid'] < 100) ][['tegevusvaldkond','Nimi','Tootajaid']]
filtered3.groupby(['tegevusvaldkond']).agg({'Nimi':'count','Tootajaid':'sum'})


,Nimi,Tootajaid
tegevusvaldkond,,
0,2,139
AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS,55,4106
EHITUS,51,3379
"ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE",7,483
FINANTS- JA KINDLUSTUSTEGEVUS,17,1285
HALDUS- JA ABITEGEVUSED,49,3390
HARIDUS,264,17862
HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT,114,7950
INFO JA SIDE,48,3351


In [40]:
#Number of employees in the enterprises: 100-249
filtered4 = taxes_data[(taxes_data['Tootajaid'] > 99) & (taxes_data['Tootajaid'] < 250) ][['tegevusvaldkond','Nimi','Tootajaid']]
filtered4.groupby(['tegevusvaldkond']).agg({'Nimi':'count','Tootajaid':'sum'})


,Nimi,Tootajaid
tegevusvaldkond,,
AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS,60,9091
EHITUS,17,2448
"ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE",4,651
FINANTS- JA KINDLUSTUSTEGEVUS,11,1678
HALDUS- JA ABITEGEVUSED,43,7157
HARIDUS,80,10312
HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT,73,10996
INFO JA SIDE,30,4542
KINNISVARAALANE TEGEVUS,6,953


In [41]:
#Number of employees in the enterprises: 250 and more
filtered5 = taxes_data[(taxes_data['Tootajaid'] > 250)][['tegevusvaldkond','Nimi','Tootajaid']]
filtered5.groupby(['tegevusvaldkond']).agg({'Nimi':'count','Tootajaid':'sum'})


,Nimi,Tootajaid
tegevusvaldkond,,
0,2,935
AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS,30,26751
EHITUS,6,1980
"ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE",4,2861
FINANTS- JA KINDLUSTUSTEGEVUS,10,6929
HALDUS- JA ABITEGEVUSED,21,15384
HARIDUS,12,11409
HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT,41,26543
INFO JA SIDE,16,9047
